In [54]:
import config
import requests
import json
import ast
import numpy as np
import pandas as pd

In [55]:
method_dict = {"GetNewsForApp":"ISteamNews/GetNewsForApp/v0002/", 
             "GetGlobalAchievementPercentagesForApp": "ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0002/", 
             "GetGlobalStatsForGame": "ISteamUserStats/GetGlobalStatsForGame/v0001/", 
             "GetPlayerSummaries": "ISteamUser/GetPlayerSummaries/v0002/", 
             "GetFriendList": "ISteamUser/GetFriendList/v0001/", 
             "GetPlayerAchievements": "ISteamUserStats/GetPlayerAchievements/v0001/", 
             "GetUserStatsForGame": "ISteamUserStats/GetUserStatsForGame/v0002/", 
             "GetOwnedGames": "IPlayerService/GetOwnedGames/v0001/", 
             "GetRecentlyPlayedGames": "IPlayerService/GetRecentlyPlayedGames/v0001/", 
             "IsPlayingSharedGame": "IPlayerService/IsPlayingSharedGame/v0001/", 
             "GetSchemaForGame": "ISteamUserStats/GetSchemaForGame/v2/", 
             "GetPlayerBans": "ISteamUser/GetPlayerBans/v1/"
            }


In [58]:
class GetOwnedGames():
    def __init__(self, steam_id):
        self.steam_id = steam_id
        fetch_url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={}&steamid={}".format(config.api_key, self.steam_id)
        self.r = requests.get(fetch_url)
        self.data = self.r.json()
        #print(json.dumps(self.data, sort_keys=True, indent=4))

In [59]:
seameat = GetOwnedGames(config.steam_user)

In [60]:
# CODE BELOW THIS POINT WILL BE INTEGRATED IN RELEVANT CLASSES
# CURRENT STATE IS SIMPLY PROOF OF CONCEPT

In [61]:
# Store total game count in variable
game_count = returned_values["response"]["game_count"]
print("Number of games owned: " + str(game_count))

# Create empty list for counting games played
my_list = []

# Identify each game with playtime over 0 minutes, and add to my_list
for i in returned_values["response"]["games"][0:-1]:
    if i["playtime_forever"] > 0:
        my_list.append(i)

# Count number of games played
game_played = len(my_list)
print("Number of games played: " + str(game_played))

# Calculate games played as a percentage of games owned
game_percent = game_played/game_count
print("Percentage of owned games that have been played: " + str(game_percent))

Number of games owned: 224
Number of games played: 126
Percentage of owned games that have been played: 0.5625


In [62]:
# Retrieving top ten most played games 
sorted_values = json.dumps(sorted(returned_values["response"]["games"], key=lambda i: i["playtime_forever"], reverse=True), indent=4)
top_ten = ast.literal_eval(sorted_values)[:10]

for i in top_ten:
    print(i)

{'appid': 49520, 'playtime_forever': 11179}
{'appid': 730, 'playtime_2weeks': 153, 'playtime_forever': 6834}
{'appid': 10190, 'playtime_forever': 4169}
{'appid': 28050, 'playtime_forever': 3331}
{'appid': 12210, 'playtime_forever': 3014}
{'appid': 241600, 'playtime_forever': 2551}
{'appid': 32370, 'playtime_forever': 2230}
{'appid': 8980, 'playtime_forever': 2116}
{'appid': 9480, 'playtime_forever': 2102}
{'appid': 212680, 'playtime_forever': 1713}


In [63]:
# Display game title for each of the top ten games
# NEXT TASK: append to list instead of printing, condense all this into a function
for i in top_ten:
    appid = i["appid"]
    appid_url = "http://store.steampowered.com/api/appdetails?appids={0}".format(appid)
    r = requests.get(appid_url)
    game_data = r.json()
    if json.dumps(game_data[str(appid)]["success"]) == "false":
        print("No data....")
    else:
        print(json.dumps(game_data[str(appid)]["data"]["name"], indent=4))
    print("-"*75)

"Borderlands 2"
---------------------------------------------------------------------------
"Counter-Strike: Global Offensive"
---------------------------------------------------------------------------
"Call of Duty\u00ae: Modern Warfare\u00ae 2"
---------------------------------------------------------------------------
No data....
---------------------------------------------------------------------------
"Grand Theft Auto IV"
---------------------------------------------------------------------------
"Rogue Legacy"
---------------------------------------------------------------------------
"STAR WARS\u2122 - Knights of the Old Republic\u2122"
---------------------------------------------------------------------------
"Borderlands"
---------------------------------------------------------------------------
"Saints Row 2"
---------------------------------------------------------------------------
"FTL: Faster Than Light"
---------------------------------------------------------------